In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import re
import locale
from time import sleep

# Define o locale para português
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

login = r'belevematriz@agoradeulucro.com'
senha = r'Agoradeulucr0.'
categoria = r'Vendas E-commerce'
planilha = r'Beleve_Amazon.csv'
amazon = pd.read_csv(planilha, skiprows=7)



Funções

In [2]:
def transformar_hora(data_str):
        if len(data_str) == 33:
            data_str = data_str[:18]
        else:
            data_str = data_str[:17]
        data_str = data_str.replace('.', '')
        sub_data = re.sub(r'(\b\d{1}\b)( de \w+ de \d{4})', r'0\1\2', data_str)
        sub_data = sub_data.strip()
        
        # Converte a string para datetime
        
        data_obj = datetime.strptime(sub_data, "%d de %b de %Y").date()
    
        data_nova = datetime.strftime(data_obj, '%d/%m/%Y')

        return data_nova
    
    
def processar_pedidos(pedidos, navegador):
    nao_encontrados = {'Pedidos':[]
                       }
    # Pré-localizar elementos fixos que não mudam com cada iteração
    barra_pesquisa = navegador.find_element(By.XPATH, '//*[@id="pesquisa-mini"]')
    botao_pesquisa_xpath = '//*[@id="container-filtros"]/div/span[1]'
    calendario_xpath = '/html/body/div[1]/div/div/div/div[2]/form/div[2]/div[2]/div/input'
    botao_receber_xpath = '/html/body/div[11]/ul/li[1]/a'
    salvar_bordero_xpath = '//*[@id="salvarBordero"]'
    
    for _, row in pedidos.iterrows():
        pedido = row['id do pedido']
        data = row['data/hora']
        
        # Limpar e inserir o pedido na barra de pesquisa
        barra_pesquisa.clear()
        barra_pesquisa.send_keys(pedido)
        navegador.find_element(By.XPATH, botao_pesquisa_xpath).click()
        sleep(5)
        
        try:
            # Verificar se a página retornou resultados
            pagina_vazia = navegador.find_element(By.XPATH, '//*[@id="page-wrapper"]/div[4]/div[2]/div[1]/div/h4')
            if 'Sua pesquisa não retornou resultados.' in pagina_vazia.text:
                nao_encontrados['Pedidos'].append(pedido)
                sleep(2)
                continue
        except NoSuchElementException:
            # Elemento não encontrado, prosseguir com as ações
            WebDriverWait(navegador, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[6]/div/div[4]/div[2]/div[2]/div/div[1]/form/table/tbody/tr/td[2]/button'))
            ).click()
            
            
            # Verifica se o botão de receber está presente e clica nele
            botao_receber = WebDriverWait(navegador, 10).until(
                EC.visibility_of_element_located((By.XPATH, botao_receber_xpath))
            )
            if botao_receber.is_displayed():
                botao_receber.click()
                sleep(3)
            
            # Selecionar "Amazon" na conta contábil
            caixa_amazon = Select(navegador.find_element(By.ID, 'idContaContabil'))
            caixa_amazon.select_by_visible_text('Amazon')
            
            # Selecionar "Vendas E-commerce" na categoria
            categoria_vendas = Select(navegador.find_element(By.ID, 'idCategoria'))
            categoria_vendas.select_by_visible_text(categoria)
            
            # Limpar e inserir a data no calendário
            calendario = navegador.find_element(By.XPATH, calendario_xpath)
            calendario.clear()
            calendario.send_keys(data)
            
            # Clicar em "Salvar"
            navegador.find_element(By.XPATH, salvar_bordero_xpath).click()
            sleep(2)
    df_naoencontrados = pd.DataFrame(nao_encontrados)
    df_naoencontrados.to_excel('Pedidos nao encontrados.xlsx', index=False)

Filtrando a planilha

In [3]:
pedidos = amazon.loc[amazon['tipo'] == 'Pedido', :]
pedidos_filtrados = pedidos[['data/hora', 'id do pedido']]
    
pedidos_filtrados['data/hora'] = pedidos_filtrados['data/hora'].apply(transformar_hora)

C:\Users\Contler Elias\AppData\Local\Temp\ipykernel_10864\1211599714.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos_filtrados['data/hora'] = pedidos_filtrados['data/hora'].apply(transformar_hora)


Fazendo login no Tiny

In [4]:
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

navegador.get(r'https://erp.tiny.com.br/contas_receber')

acesso_login = login
acesso_senha = senha

try:
    sleep(5)
    alerta = navegador.switch_to.alert
    if alerta:
        alerta.accept()   
    login = navegador.find_element(By.XPATH, '//*[@id="kc-content-wrapper"]/react-login/div/div/div[1]/div[1]/div[1]/form/div[1]/div/input').send_keys(acesso_login)
    senha = navegador.find_element(By.XPATH, '//*[@id="kc-content-wrapper"]/react-login/div/div/div[1]/div[1]/div[1]/form/div[2]/div/input').send_keys(acesso_senha)
    click = navegador.find_element(By.XPATH, '//*[@id="kc-content-wrapper"]/react-login/div/div/div[1]/div[1]/div[1]/form/div[3]/button').click()
except:
    print('Elemento nao encontrado a tempo')
    
    

Filtrando o periodo no Tiny

In [5]:
try:
    sleep(5)
    periodo = navegador.find_element(By.XPATH, '//*[@id="periodoDatas"]').click()
    periodo_semfiltro = navegador.find_element(By.XPATH, '//*[@id="opc-todas"]').click()
    filtro = navegador.find_element(By.XPATH, '//*[@id="container-filtros"]/div/span[2]/button/span[1]/i').click()
    filtro_historico = navegador.find_element(By.XPATH, '//*[@id="container-filtros"]/div/span[2]/ul/li[5]/a').click()
except:
    print('Não encontrados')

Realizando o laço no lançamento de contas

In [6]:
processar_pedidos(pedidos_filtrados, navegador)  